In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split

from tensorflow.python.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPool1D
import tensorflow as tf

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession



In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
df = pd.read_csv('../data/raw_recording.csv')
df

,x_pos,y_pos,channel_1,channel_2,channel_3,channel_4
0,2491.0,1219.0,0,0,0,0
1,2491.0,1219.0,551,86,24,14
2,2489.0,1217.0,120,86,24,19
3,2488.0,1216.0,120,86,24,25
4,2484.0,1214.0,120,85,24,26
...,...,...,...,...,...,...
85060,2235.0,643.0,115,60,18,35
85061,2234.0,643.0,115,60,18,36
85062,2234.0,643.0,115,60,17,36
85063,2233.0,642.0,114,60,17,35


In [4]:
# df.drop(columns=['time', 'start', 'end'], inplace=True)

In [5]:
for i in range(1,500):

    df[f'channel_1_{i}'] = df['channel_1'].shift(i)
    df[f'channel_2_{i}'] = df['channel_2'].shift(i)
    df[f'channel_3_{i}'] = df['channel_3'].shift(i)
    df[f'channel_4_{i}'] = df['channel_4'].shift(i)

In [6]:
df['x_diff'] = df['x_pos'].diff()
df['y_diff'] = df['y_pos'].diff()

In [7]:
df.dropna(inplace=True)

In [8]:
df.reset_index(drop=True).head()

,x_pos,y_pos,channel_1,channel_2,channel_3,channel_4,channel_1_1,channel_2_1,channel_3_1,channel_4_1,...,channel_1_498,channel_2_498,channel_3_498,channel_4_498,channel_1_499,channel_2_499,channel_3_499,channel_4_499,x_diff,y_diff
0,1740.0,1038.0,111,302,39,16,111.0,302.0,39.0,16.0,...,551.0,86.0,24.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1736.0,1038.0,111,302,39,16,111.0,302.0,39.0,16.0,...,120.0,86.0,24.0,19.0,551.0,86.0,24.0,14.0,-4.0,0.0
2,1731.0,1037.0,110,301,39,16,111.0,302.0,39.0,16.0,...,120.0,86.0,24.0,25.0,120.0,86.0,24.0,19.0,-5.0,-1.0
3,1731.0,1037.0,111,301,39,16,110.0,301.0,39.0,16.0,...,120.0,85.0,24.0,26.0,120.0,86.0,24.0,25.0,0.0,0.0
4,1728.0,1036.0,110,300,39,16,111.0,301.0,39.0,16.0,...,120.0,86.0,24.0,26.0,120.0,85.0,24.0,26.0,-3.0,-1.0


In [9]:
# df = df[df['x_diff'] != 0]
# df = df[df['y_diff'] != 0]

In [10]:
# print((df['x_diff'].value_counts()[0], sum(df['x_diff'].value_counts().reset_index(drop = True)[1:])))
# df['y_diff'].value_counts()[0], sum(df['y_diff'].value_counts().reset_index(drop = True)[1:])

In [11]:
# df = pd.concat([df, df[df['x_diff'] != 0].sample(52201 - 32365, replace = True)], axis=0)
# df = pd.concat([df, df[df['y_diff'] != 0].sample(52201 - 32365, replace = True)], axis=0)

In [12]:
X = df.drop(columns=['x_pos', 'y_pos', 'x_diff', 'y_diff'])
y = np.array(df['x_diff'])

W = X.copy()
z = np.array(df['y_diff'])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
W_train, W_test, z_train, z_test = train_test_split(W, z, random_state = 42)

In [14]:
model_x = Sequential()


# model_x.add(Conv1D(512, kernel_size=5, input_shape = (2000,1), activation='relu'))
# model_x.add(GlobalMaxPool1D())

model_x.add(Dense(128, activation='relu', input_shape = (2000,)))
model_x.add(Dense(32, activation = 'relu'))
model_x.add(Dense(1, activation=tf.keras.activations.linear))

model_x.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['acc'])

In [15]:
model_x.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 20)

Train on 63424 samples, validate on 21142 samples
Epoch 1/20
63424/63424 [==============================] - 3s 44us/sample - loss: 799.2383 - acc: 0.5691 - val_loss: 13.0713 - val_acc: 0.6129
Epoch 2/20
63424/63424 [==============================] - 3s 41us/sample - loss: 16.9772 - acc: 0.6165 - val_loss: 11.9552 - val_acc: 0.6142
Epoch 3/20
63424/63424 [==============================] - 3s 41us/sample - loss: 18.7157 - acc: 0.6173 - val_loss: 11.8995 - val_acc: 0.6141
Epoch 4/20
63424/63424 [==============================] - 3s 41us/sample - loss: 12.4728 - acc: 0.6182 - val_loss: 11.8670 - val_acc: 0.6144
Epoch 5/20
63424/63424 [==============================] - 3s 41us/sample - loss: 12.3097 - acc: 0.6182 - val_loss: 11.8675 - val_acc: 0.6144
Epoch 6/20
63424/63424 [==============================] - 3s 41us/sample - loss: 12.3097 - acc: 0.6182 - val_loss: 11.8665 - val_acc: 0.6144
Epoch 7/20
63424/63424 [==============================] - 3s 41us/sample - loss: 12.3099 - acc: 0.6182 

In [16]:
model_y = Sequential()

model_y.add(Dense(128, input_shape = (2000,), activation='relu'))
model_y.add(Dense(32, activation = 'relu'))
model_y.add(Dense(1, activation=tf.keras.activations.linear))

model_y.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['acc'])


In [17]:
model_y.fit(W_train, z_train, validation_data = (W_test, z_test), epochs = 20)

Train on 63424 samples, validate on 21142 samples
Epoch 1/20
63424/63424 [==============================] - 3s 45us/sample - loss: 283.0481 - acc: 0.6051 - val_loss: 6.3085 - val_acc: 0.6453
Epoch 2/20
63424/63424 [==============================] - 3s 41us/sample - loss: 39.9871 - acc: 0.6431 - val_loss: 5.9073 - val_acc: 0.6463
Epoch 3/20
63424/63424 [==============================] - 3s 41us/sample - loss: 33.2127 - acc: 0.6457 - val_loss: 5.8216 - val_acc: 0.6470
Epoch 4/20
63424/63424 [==============================] - 3s 41us/sample - loss: 17.1755 - acc: 0.6462 - val_loss: 5.8213 - val_acc: 0.6470
Epoch 5/20
63424/63424 [==============================] - 3s 41us/sample - loss: 17.1754 - acc: 0.6462 - val_loss: 5.8220 - val_acc: 0.6470
Epoch 6/20
63424/63424 [==============================] - 3s 41us/sample - loss: 17.1754 - acc: 0.6462 - val_loss: 5.8207 - val_acc: 0.6470
Epoch 7/20
63424/63424 [==============================] - 3s 41us/sample - loss: 17.1755 - acc: 0.6462 - val_

In [ ]:
plt.figure(figsize=(100,100))
plt.plot(y, z);
plt.plot(model_x.predict(X), model_y.predict(W));


In [ ]:
plt.figure(figsize=(100,100))
plt.plot(y);
plt.plot(model_x.predict(X));


In [ ]:
plt.figure(figsize=(100,100))
plt.plot(z);
plt.plot(model_x.predict(W));


In [ ]:
model_x.save('../pickled_models/nn_x')
model_y.save('../pickled_models/nn_y')

In [14]:
q = np.array(df[['x_pos', 'y_pos']])

P_train, P_test, q_train, q_test = train_test_split(X, q, random_state = 42)

In [15]:
model = Sequential()

model.add(Dense(128, input_shape = (2000,), activation='relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(2, activation=tf.keras.activations.linear))

model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['acc'])


In [16]:
model.fit(P_train, q_train, validation_data = (P_test, q_test), epochs = 1000)

Train on 63424 samples, validate on 21142 samples
Epoch 1/1000
63424/63424 [==============================] - 3s 45us/sample - loss: 513359.2366 - acc: 0.8877 - val_loss: 445607.9853 - val_acc: 0.8870
Epoch 2/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 391354.2949 - acc: 0.8883 - val_loss: 383173.8352 - val_acc: 0.8849
Epoch 3/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 329613.8986 - acc: 0.8887 - val_loss: 345535.4367 - val_acc: 0.8852
Epoch 4/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 287580.7344 - acc: 0.8900 - val_loss: 316877.7422 - val_acc: 0.8855
Epoch 5/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 258128.6092 - acc: 0.8914 - val_loss: 304781.6056 - val_acc: 0.8874
Epoch 6/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 236578.8390 - acc: 0.8937 - val_loss: 284405.2699 - val_acc: 0.8885
Epoch 7/1000
63424/63424 [==================

Epoch 55/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 64127.3548 - acc: 0.9314 - val_loss: 352945.2734 - val_acc: 0.8985
Epoch 56/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 64097.5780 - acc: 0.9308 - val_loss: 366121.4318 - val_acc: 0.9028
Epoch 57/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 62088.4563 - acc: 0.9321 - val_loss: 354193.0124 - val_acc: 0.9026
Epoch 58/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 63230.2791 - acc: 0.9314 - val_loss: 354715.6877 - val_acc: 0.9022
Epoch 59/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 61661.4751 - acc: 0.9320 - val_loss: 337583.1079 - val_acc: 0.8966
Epoch 60/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 61775.5633 - acc: 0.9324 - val_loss: 363520.0232 - val_acc: 0.8987
Epoch 61/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 60755.1140

Epoch 163/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 36974.1109 - acc: 0.9491 - val_loss: 380633.6863 - val_acc: 0.9037
Epoch 164/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 36870.1952 - acc: 0.9502 - val_loss: 377894.9070 - val_acc: 0.9081
Epoch 165/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 37071.7465 - acc: 0.9502 - val_loss: 400604.9243 - val_acc: 0.9051
Epoch 166/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 37033.1898 - acc: 0.9504 - val_loss: 393034.3653 - val_acc: 0.9115
Epoch 167/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 36756.0519 - acc: 0.9495 - val_loss: 414836.2022 - val_acc: 0.9103
Epoch 168/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 36679.1568 - acc: 0.9498 - val_loss: 382148.4464 - val_acc: 0.9074
Epoch 169/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 361

Epoch 217/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 32466.1051 - acc: 0.9527 - val_loss: 430527.6491 - val_acc: 0.9116
Epoch 218/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 32547.3549 - acc: 0.9525 - val_loss: 416232.7488 - val_acc: 0.9064
Epoch 219/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 32420.7391 - acc: 0.9527 - val_loss: 425897.6454 - val_acc: 0.9026
Epoch 220/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 32040.8286 - acc: 0.9532 - val_loss: 395423.1576 - val_acc: 0.9064
Epoch 221/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 32266.5106 - acc: 0.9532 - val_loss: 429123.8431 - val_acc: 0.9034
Epoch 222/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 32127.7589 - acc: 0.9533 - val_loss: 422661.3402 - val_acc: 0.9086
Epoch 223/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 324

Epoch 271/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 29254.1616 - acc: 0.9568 - val_loss: 435869.3950 - val_acc: 0.9066
Epoch 272/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 29715.6339 - acc: 0.9554 - val_loss: 445996.1436 - val_acc: 0.9109
Epoch 273/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 29311.6417 - acc: 0.9566 - val_loss: 433491.9882 - val_acc: 0.9081
Epoch 274/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 29296.5558 - acc: 0.9560 - val_loss: 440354.5260 - val_acc: 0.9085
Epoch 275/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 29117.2516 - acc: 0.9562 - val_loss: 436757.7100 - val_acc: 0.9116
Epoch 276/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 29147.5410 - acc: 0.9556 - val_loss: 435497.5924 - val_acc: 0.9076
Epoch 277/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 291

...

Epoch 757/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19456.0880 - acc: 0.9657 - val_loss: 570451.4449 - val_acc: 0.9151
Epoch 758/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19797.9916 - acc: 0.9651 - val_loss: 550974.1105 - val_acc: 0.9139
Epoch 759/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19386.9358 - acc: 0.9649 - val_loss: 570460.2615 - val_acc: 0.9157
Epoch 760/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19706.7102 - acc: 0.9647 - val_loss: 583260.8956 - val_acc: 0.9140
Epoch 761/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 19711.2776 - acc: 0.9654 - val_loss: 541917.0056 - val_acc: 0.9125
Epoch 762/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19701.4623 - acc: 0.9652 - val_loss: 533225.2608 - val_acc: 0.9105
Epoch 763/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 194

Epoch 811/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19476.1257 - acc: 0.9659 - val_loss: 581673.9639 - val_acc: 0.9090
Epoch 812/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19021.0171 - acc: 0.9656 - val_loss: 557736.3045 - val_acc: 0.9126
Epoch 813/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19004.1386 - acc: 0.9659 - val_loss: 560698.7720 - val_acc: 0.9033
Epoch 814/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 18594.0720 - acc: 0.9665 - val_loss: 610176.5815 - val_acc: 0.9139
Epoch 815/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19058.3493 - acc: 0.9666 - val_loss: 596637.0782 - val_acc: 0.9098
Epoch 816/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 19167.8279 - acc: 0.9660 - val_loss: 564247.5683 - val_acc: 0.9131
Epoch 817/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 190

Epoch 865/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18717.4715 - acc: 0.9655 - val_loss: 572024.1044 - val_acc: 0.9100
Epoch 866/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18614.2495 - acc: 0.9661 - val_loss: 609660.5732 - val_acc: 0.9086
Epoch 867/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18715.8012 - acc: 0.9661 - val_loss: 583193.4676 - val_acc: 0.9084
Epoch 868/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18500.4357 - acc: 0.9661 - val_loss: 585737.3378 - val_acc: 0.9085
Epoch 869/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18387.4223 - acc: 0.9672 - val_loss: 587011.2630 - val_acc: 0.9078
Epoch 870/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18528.9935 - acc: 0.9669 - val_loss: 600359.3908 - val_acc: 0.9054
Epoch 871/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 189

Epoch 919/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 18337.8959 - acc: 0.9672 - val_loss: 603100.0977 - val_acc: 0.9055
Epoch 920/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18538.1176 - acc: 0.9672 - val_loss: 649818.3772 - val_acc: 0.9121
Epoch 921/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18355.9514 - acc: 0.9670 - val_loss: 613989.4727 - val_acc: 0.9070
Epoch 922/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18225.9467 - acc: 0.9662 - val_loss: 601955.3102 - val_acc: 0.9101
Epoch 923/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18049.0997 - acc: 0.9665 - val_loss: 607691.4505 - val_acc: 0.8978
Epoch 924/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18402.4630 - acc: 0.9680 - val_loss: 620548.8654 - val_acc: 0.9142
Epoch 925/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 179

Epoch 973/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18036.0994 - acc: 0.9659 - val_loss: 625878.0793 - val_acc: 0.9097
Epoch 974/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 17859.3625 - acc: 0.9672 - val_loss: 621376.4089 - val_acc: 0.9054
Epoch 975/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 17959.7223 - acc: 0.9667 - val_loss: 637301.7030 - val_acc: 0.9122
Epoch 976/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 18223.8382 - acc: 0.9667 - val_loss: 580009.8946 - val_acc: 0.9102
Epoch 977/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 17801.1872 - acc: 0.9672 - val_loss: 650783.7068 - val_acc: 0.9101
Epoch 978/1000
63424/63424 [==============================] - 3s 40us/sample - loss: 18045.3408 - acc: 0.9675 - val_loss: 663774.5124 - val_acc: 0.9119
Epoch 979/1000
63424/63424 [==============================] - 3s 41us/sample - loss: 180

In [ ]:
plt.figure(figsize=(100,100))

plt.plot(model.predict(X)[:,0][:10000])
plt.plot(q[:,0][:10000])

In [ ]:
plt.figure(figsize=(100,100))

plt.plot(model.predict(X)[:,1][:10000])
plt.plot(q[:,1][:10000])

In [ ]:
plt.figure(figsize=(100,100))

plt.plot(model.predict(X)[:,1][:1000])
plt.plot(model.predict(X)[:,0][:1000])

In [ ]:
plt.figure(figsize=(100,100))

plt.plot(q[:,1][:1000])
plt.plot(q[:,0][:1000])

In [ ]:



ploty = pd.DataFrame({'x_pred':model.predict(X)[:,0], 'y_pred':model.predict(X)[:,1], 'x_diff':df['x_diff'], 
                    'y_diff':df['y_diff']})

In [ ]:
ploty.head()

In [ ]:
ploty['x_pred_dif'] = ploty['x_pred'].diff()
ploty['y_pred_dif'] = ploty['y_pred'].diff()
ploty.head()

In [ ]:
ploty.dropna(inplace=True)
ploty.reset_index(drop=True)

In [ ]:
plt.figure(figsize=(100,100))
plt.plot(ploty['x_pred_dif'][:1000])
plt.plot(ploty['x_diff'][:1000])

In [ ]:
plt.figure(figsize=(100,100))
plt.plot(ploty['y_pred_dif'][:1000])
plt.plot(ploty['y_diff'][:1000])